In [2]:
import os
import numpy as np


float_precision = 'float32'
root_file = os.getcwd()
dim, mesh_path = (3, 'tetrahedron.msh')
mesh_path = os.path.join(root_file, 'mesh', mesh_path)

def get_members(a):
  return [m for m in dir(a) if not m.startswith("__")]


In [2]:
import importlib

module = importlib.import_module("create_domain")
importlib.reload(module)
Mesh = getattr(module, "Mesh")

mesh = Mesh(mesh_path, dim)

In [ ]:
# Measure Time

import os
import numpy as np


float_precision = 'float32'
root_file = os.getcwd()
dim, mesh_path = (3, 'tetrahedron.msh')
mesh_path = os.path.join(root_file, 'mesh', mesh_path)

def get_members(a):
  return [m for m in dir(a) if not m.startswith("__")]

import importlib

module_1 = importlib.import_module("create_domain_utils")
importlib.reload(module_1)
module = importlib.import_module("create_domain")
importlib.reload(module)
Mesh = getattr(module, "Mesh")
Domain = getattr(module, "Domain")
LocalDomain = getattr(module, "LocalDomain")

mesh = Mesh(mesh_path, dim)
domain = Domain(mesh, float_precision)
local_domains_data = domain.c_create_sub_domains(4)
# new_nb_part = len(local_domains_data)

# PC
# 2m 2s -> 30000 (nid)
# 37s -> 4 (nid)
# c 10s -> 4 (fid)
# c 40s -> 30000 (fid)
# c 1m 42s -> 30000 (nid)
# ---- laptop
# 1m 44s -> 4 (nid)
# print(local_domains_data)
# --- tobkal
# 3m 8s -> 48000000 (3000, fid)





node_cellid
Execution time: 2.476237 seconds
cell_cellnid


In [9]:
import importlib

module_1 = importlib.import_module("create_domain_utils")
importlib.reload(module_1)
module = importlib.import_module("create_domain")
importlib.reload(module)
Mesh = getattr(module, "Mesh")
Domain = getattr(module, "Domain")
LocalDomain = getattr(module, "LocalDomain")

mesh = Mesh(mesh_path, dim)
domain = Domain(mesh, float_precision)
local_domains_data = domain.c_create_sub_domains(4)




node_cellid
Execution time: 0.047022 seconds
cell_cellnid
Execution time: 2.075791 seconds
cell_cellfid
Execution time: 3.060432 seconds
Start creating sub domains
Parse Input 
Start Creating SubDomains 
Make n part
Start dense to csr
Nb_vertices = 6000
size = 22800
End dense to csr
End METIS_PartGraphKway
End function
Create local cells
Create Physical faces
Create local domains struct
Create local domains struct p=0
Cells, CellsType, CellsLocToGlob
PhyFaces, PhyFacesName
Halo neighsub, Halos
End Halo neighsub, Halos
Create local domains struct p=1
Cells, CellsType, CellsLocToGlob
PhyFaces, PhyFacesName
Halo neighsub, Halos
End Halo neighsub, Halos
Create local domains struct p=2
Cells, CellsType, CellsLocToGlob
PhyFaces, PhyFacesName
Halo neighsub, Halos
End Halo neighsub, Halos
Create local domains struct p=3
Cells, CellsType, CellsLocToGlob
PhyFaces, PhyFacesName
Halo neighsub, Halos
End Halo neighsub, Halos
End function
Return List 
Execution time: 3.071522 seconds


In [6]:
print(local_domains_data[1].max_node_haloid)
print(local_domains_data[0].halo_halosext.shape)
print(local_domains_data[0].node_haloid.shape)

22
(839,)


AttributeError: 'LocalDomainStruct' object has no attribute 'node_haloid'

In [10]:
L1 = LocalDomain(local_domains_data[0])

node_cellid
Execution time: 0.011053 seconds
cell_cellnid
Execution time: 0.471592 seconds
_create_info
Execution time: 0.734231 seconds
Create cell volume and center
Execution time: 0.742986 seconds
Face measure face center face normal
Execution time: 0.773413 seconds
cell_halofid, cell_halonid, face_haloid, node_haloid
Execution time: 0.818268 seconds
Create face and node names
Execution time: 0.830575 seconds


In [16]:
a = L1.node_name
b = L1.node_oldname

print(a[:100])
print(b[:100])

[10 10 10  3  6  0  0  3  3  6  0  3  3  6  0  2  2  2  2 10  6  0  0  6
  0  6  0  6  0  2  2 10  6  0  0  6  0  6  0  6  0  2  2 10  6  0  0  6
  0  6  0  6  0  6  0  2  2 10 10  0  0  6  0  6  0  6  0  6  0  2 10 10
  0  0  6  0 10  0 10  0 10 10 10 10 10 10  0  0  3  0  3  0  2  2  0  0
  0  0  0  2]
[3 6 3 3 6 0 0 3 3 6 0 3 3 6 0 2 2 2 2 6 6 0 0 6 0 6 0 6 0 2 2 6 6 0 0 6 0
 6 0 6 0 2 2 6 6 0 0 6 0 6 0 6 0 6 0 2 2 6 6 0 0 6 0 6 0 6 0 6 0 2 2 6 0 0
 6 0 6 0 6 0 6 2 6 6 3 3 0 0 3 0 3 0 2 2 0 0 0 0 0 2]


In [3]:
from manapy.partitions import MeshPartition
from manapy.base.base import Struct

running_conf = Struct(backend="numba", signature=True, cache=True, float_precision=float_precision)
a = MeshPartition(mesh_path, dim=dim, conf=running_conf, periodic=[0,0,0])

# 6m 49 -> 30000
# 2m -> 3000
# 28s -> 4
# ----
# 1m 26s -> 4


Reading gmsh file ...
Saving partition files ...
Number of Cells: 6000
Number of Vertices: 1331


In [4]:
from manapy.ddm import Domain
domain = Domain(dim=dim, conf=running_conf)

Local domain contruction ...
Start ...
_compute_cells_info`: 0.013672 seconds
_make_neighbors: 0.042247 seconds
_define_eltypes: 0.042749 seconds
_create_faces_cons: 0.069024 seconds
Create info 3dfaces ...
oriente_3dfacenodeid ...
E_define_face_info: 0.099967 seconds
_make_halo_info: 0.108881 seconds
_update_boundaries: 0.109152 seconds
_update_ghost_info: 0.120026 seconds
_prepare_communication: 0.120349 seconds
_update_periodic_boudaries: 0.120408 seconds
_update_haloghost: 0.139847 seconds
_compute_diamondCell_info: 0.168605 seconds
_compute_orthocenter: 0.168679 seconds


In [34]:
print(domain.nodes.ghostcenter[1])
print(domain.nodes.ghostid[1])

[[ 1.025e+01  2.500e-01  3.750e-01  5.403e+03  4.000e+00  7.000e+00]
 [ 9.250e+00  1.250e-01 -3.750e-01  5.400e+03  1.000e+00  1.740e+02]
 [ 9.750e+00  2.500e-01 -3.750e-01  5.403e+03  1.000e+00  1.750e+02]
 [ 9.250e+00 -1.250e-01  3.750e-01  5.400e+03  3.000e+00  6.140e+02]
 [ 9.500e+00 -1.250e-01  7.500e-01  5.401e+03  3.000e+00  6.150e+02]
 [-1.000e+00 -1.000e+00 -1.000e+00 -1.000e+00 -1.000e+00 -1.000e+00]]
[  7 174 175 614 615  -1   5]


In [5]:
def get_members(a):
    for m in dir(a):
        try:
            if not m.startswith("__") and isinstance(getattr(a, m), np.ndarray):
                yield m
        except:
            pass

for item in get_members(domain.faces):
    print(item)

_airDiamond
_binormal
_cellid
_center
_dist_ortho
_f_1
_f_2
_f_3
_f_4
_ghostcenter
_halofid
_mesure
_name
_nodeid
_normal
_oldname
_param1
_param2
_param3
_param4
_tangent
airDiamond
binormal
cellid
center
dist_ortho
f_1
f_2
f_3
f_4
ghostcenter
halofid
mesure
name
nodeid
normal
oldname
param1
param2
param3
param4
tangent
